In [3]:
%cd ..

C:\Users\kubak\Desktop\PhiloModel\github\AgoraRAG


C:\Users\kubak\AppData\Local\Programs\Python\Python312\Lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
import os
import openai
import sys
from dotenv import load_dotenv, find_dotenv

In [1]:
sys.path.append('../..')

_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.environ['OPENAI_API_KEY']

# Load data

In [4]:
from my_code.load_data.import_pdfs import load_pdfs_from_dir

In [5]:
dir_path = f"../../data/pdfs/books/HistoriaFilozofii-Tatarkiewicz/"

In [6]:
all_pdfs = load_pdfs_from_dir(dir_path)

Processing ../../data/pdfs/books/HistoriaFilozofii-Tatarkiewicz\wadysaw-tatarkiewicz-historia-filozofii-tom1.pdf
Processing ../../data/pdfs/books/HistoriaFilozofii-Tatarkiewicz\wadysaw-tatarkiewicz-historia-filozofii-tom2.pdf
Loaded 443 docs.


# Splitting data

In [ ]:
#!pip install chromadb

In [7]:
# Split
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1500,
    chunk_overlap = 150
)

In [9]:
splits = text_splitter.split_documents(all_pdfs)

In [10]:
len(splits)

1644

In [16]:
splits[100].page_content

'"żywiołów" i "pierwiastków". Jak z niewielu barw na palecie malarza powstaje barwny obraz, tak  \nteż  z  niewielu  żywiołów  powstają  różnorodne  rzeczy.  Empedokles  nie  rozwiązał  wprawdzie \nzagadnienia szczęśliwie, ale postawił je trafnie. Zaczął bowiem szukać prostych składników materii \ni może być uważany za twórcę pojęcia pierwiastka. Rozmnożywszy Parmenideiskie pojęcie bytu,  \npotrafił uzgodnić je ze zjawiskami i uczynić zastosowalnym w przyrodoznawstwie.\nPrzygotował  w  ten  sposób  podstawę  do  rozważań  natury  chemicznej.  Sam  rozwa-żania  te \nzapoczątkował: próbował tłumaczyć wzajemne zachowanie się ciał przez ich po-winowactwo, a  \nróżnice między ciałami traktował jako różnice ilościowe i różnice struktury. Zastosowania były  \nnaiwne (mówił np., że mięso i krew mają równe ilości żywiołów, kość jest zaś w połowie ogniem,  \na w jednej czwartej ziemią i w jednej czwartej wodą), jednakże zawierały w zaczątkowej postaci  \nzasadnicze myśli chemii.\n2. TEORIA SIŁ. 

# Embedding

In [22]:
from langchain.embeddings.openai import OpenAIEmbeddings
embedding = OpenAIEmbeddings()

C:\Users\kubak\AppData\Local\Temp\ipykernel_21496\1742550774.py:2: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding = OpenAIEmbeddings()


# Vectorstores

In [19]:
from langchain.vectorstores import Chroma

In [20]:
persist_directory = 'docs/chroma/'

In [ ]:
#!rm -rf ./docs/chroma  # remove old database files if any

In [23]:
vectordb = Chroma.from_documents(
    documents=splits,
    embedding=embedding,
    persist_directory=persist_directory
)

In [24]:
print(vectordb._collection.count())

1644


# Similiarity Search

In [30]:
question = "Kim by Kant?"

In [31]:
docs = vectordb.similarity_search(question,k=3)

In [32]:
docs[0].page_content

'intelektualnych Niemiec przez ziemie polskie i zdanym na siebie. Uniwersytet Królewiecki był \ndrugorzędną  uczelnią,  która  dopiero  dzięki  Kantowi  zyskała  rozgłos;  nie  dane  było  Kantowi \nobcować  z  ludźmi,  którzy  byliby  równi  lub  choćby  zbliżeni  doń  poziomem  umysłu.  W \nprzeciwieństwie do innych, doniosłych wydarzeń w dziejach filozofii, które dokonywały się w \nświetnych środowiskach umysłowych i w epokach obfitujących w umysły twórcze - czy to były \nAteny w V i VI wieku, czy Londyn początku XVIII w., czy Paryż połowy XVIII w. - tu reforma \nfilozofii dokonana została przez samotnego człowieka na głuchej prowincji.\nTrudne  warunki,  obciążenie  pracą  i  wątłość  zdrowia  Kant  neutralizował  niezwyczajnie \nekonomicznym trybem życia. Było to życie przysłowiowo regularne, systematyczne i do-browolnie \nnakładające sobie ograniczenia. Żył sam, nie założył rodziny. Instynktownie odrzucał powołania do \ninnych uniwersytetów (do Erlangen, Halle, Jeny) które rokowa